## Topic models with Non-negative Matrix Factorization (NMF)

unsupervised search for classes of terms (words) that form a topic given a particular text corpus

methods: NMF and LDA (for comparison)

use TuebaD/Z (German newspaper): only nouns are being used

### Content

* topic modelling of a newspaper corpus: small data set (first)
     * vectorize and factorize the data
* use the models
    * similarity of documents
    * similarity of words
* topic modelling of a newspaper corpus: full data
    * visualize and explore the model with pyLDAvis
* extractive summarization using the topic model
    * find the 5 most important sentences of a document given its topic
    * find the most important speech given a topic


### Explorative Setting

* 6 documents (documents=documents[:6])
* 10 words uses as features (no_features = 10)
* number of topics: 4 (no_topics = 4)
* number of words used to characterise topics: 5 (no_top_words=5)

* M = 6x10, W=6x4, H=4x10
* M=WxH, M = documents x words, W = documents x topics, H = topics x words
* H = word space, W = document space


In [ ]:
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/lemmaNoun.tueba -O lemmaNoun.tueba
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/sp.tsv -O sp.tsv
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/svp.tsv -O svp.tsv
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/blocher.tsv -O blocher.tsv
! wget https://files.ifi.uzh.ch/cl/siclemat/lehre/hs19/tm/doc.tueba -O doc.tueba

In [ ]:
# Get the documents (list of lemmata from the TuebaD/Z)

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

exec(open("lemmaNoun.tueba").read())  # this is a file with: documents=['doc1',...]

documents6=documents[:6]  # just 6 documents 
documents[0]

def removeDocs(documents):
    """ remove too small documents, helps pyLDAvis"""
    doc=[]

    for i in range(0,len(documents)-1):
        if len(documents[i]) <250 :
            continue
        else:
            doc.append(documents[i])

    return doc

documents=removeDocs(documents)
documents[0]

In [ ]:
no_features = 10

# we get a matrix with tifidf cell entries: 6 rows (documents) and |V| columns (vocabulary)
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(documents6)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tfidf_feature_names

* max_df=skip words if they are in 95% of the documents
* min_df=only words that occur 1 times or more
* max_features= build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

### Factorize

no_topics = number of topics

In [ ]:
no_topics = 4

# Instantiate NMF, the number of columns of W will be 4, whereas H has 4 lines
nmf = NMF(n_components=no_topics, solver='cd',beta_loss=2, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

numpy argsort: sorts values (increasing) and return there index positions

np.array([1.2,1.4,4.3]).argsort()  produces: array([0, 1, 2])

In [ ]:
def display_topics(model, feature_names, no_top_words):
    """ each line of H is a topic, we get the indexes of the strongest weighted word and print them from feature_names"""
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
# we fit W, H comes for free

W=nmf.fit_transform(tfidf)
H=nmf.components_

In [ ]:
W  # documents space, no probabilities, just weights; document 1 has (best) topic 0

In [ ]:
H # e.g. topic 0's most important term (0.755) is number 4  which is 'daewoo'

In [ ]:
tfidf_feature_names[4]

In [ ]:
no_top_words=5

display_topics(nmf, tfidf_feature_names, no_top_words)

### Topic of a document, similarity of documents

In [ ]:
def getTopic(X,H,no_topics):
    """ which topic is most similar to X (a tfidf representation of a document)"""
    bestsim=0
    for i in range(0,no_topics):
        sim=np.dot(X,H[i])/(norm(X)*norm(H[i]))
        if sim>bestsim:
            bestsim=sim
            index=i
    return index

In [ ]:
# we could (should) use a new document, but for simplicity, we use an existing on

from numpy.linalg import norm

X=tfidf[2].toarray()     # 3. document as tfidf 
Y=tfidf[3].toarray()     # 4. document

t3=getTopic(X,H,no_topics)
t4=getTopic(Y,H,no_topics)
t3,t4


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# which is the document closest to doc 2 based on the topic representation
# just compare to the rows of W with cosine 

bestsim=0
for i in range(0,len(documents6)):
    if i==2:
        continue
    else:
        sim=np.dot(W[2],W[i])/(norm(W[2])*norm(W[i]))
        if sim>bestsim:
            bestsim=sim
            index=i

bestsim,index


### Similarity of words

just compare the columns from H with cosine (to which topic with which degree belongs the word)

In [ ]:
tfidf_feature_names

In [ ]:
H  # dimension 2 and 3: topic of 'awo' and 'bremerhafen'

In [ ]:
H[:,2]  # access to column 0

In [ ]:
np.dot(H[:,4],H[:,3])/(norm(H[:,4])*norm(H[:,3]))  # 'daewoo' and 'bremerhafen'

In [ ]:
np.matmul(W,H),tfidf.todense()  # reconstruct orignial tfidf matrix

In [ ]:
# topic 2: 1,6-10]

H[2] # tranform 2. topic into 5 top words

In [ ]:
# awo wedemeier landesverband geschäftsführer ute
tfidf_feature_names[0:1] + tfidf_feature_names[5:6] +tfidf_feature_names[7:10]

### topic model for the whole corpus

In [ ]:
no_features=500

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features)

dtm_tfidf = tfidf_vectorizer.fit_transform(documents)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA
#tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
#tf = tf_vectorizer.fit_transform(documents)
#tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
no_topics = 20

# Run NMF
nmf_tfidf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(dtm_tfidf)
#nmf2 = NMF(n_components=no_topics, solver='mu',random_state=1,beta_loss='kullback-leibler').fit(tfidf)

# Run LDA
#lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [ ]:
#### from sklearn.decomposition import NMF
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

#pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

vis_nmf = pyLDAvis.sklearn.prepare(nmf_tfidf, dtm_tfidf, tfidf_vectorizer)
vis_nmf

### Application Topic Modelling: Extractive Summarization

means: select the n most important sentences from a document

most important:  sentences best representing the topic of a text

In [ ]:
exec(open("lemmaNoun.tueba").read())

In [ ]:
no_features=5000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, max_features=no_features)

tfidf = tfidf_vectorizer.fit_transform(documents)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
no_topics = 50

nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [ ]:
no_top_words=30

display_topics(nmf, tfidf_feature_names, no_top_words)

In [ ]:
W=nmf.fit_transform(tfidf)
H=nmf.components_

In [ ]:
documents[0]

In [ ]:
X=tfidf[:1].toarray()  # tfidf representation of document 1

topicIndex=getTopic(X,H,no_topics)  # get the topic of document 1 (its index)
topicIndex

In [ ]:
exec(open("doc.tueba").read())   # doc=[sentence1,....] von segment 1

# find the n best sentences (out of 33) of document 1 given the topic of document 1

def bestSentencesOfTopic(doc,H,index,lines):
    """ lines=number of sentences 
        index=topic index
    """
    simIndexed={}
    for i in range(0,lines):
        X=tfidf_vectorizer.transform([doc[i]])
        X=X[0].todense()
        if norm(X)==0:
            simIndexed[i]=0
            continue
        sim=np.dot(X,H[index])/(norm(X)*norm(H[index]))
        simIndexed[i]=np.array(sim)[0].tolist()[0]  
    return simIndexed
    
simIndexed= bestSentencesOfTopic(doc,H,topicIndex,33)

simIndexed
i=sorted([(value,key) for (key,value) in simIndexed.items()])
i[-6:]

sentences 2,7,10,19,20,22 are the best


Summary: 6 sentences

Flossen 165.000 Mark Sammelgelder für Flutopfer in ein Altenheim in Danzig ? 
Vorwurf Nummer 1 : 165.000 Mark aus der bundesweiten Geldsammlung für die Flutopfer in Südpolen seien über das Konto des Bremer Landesverbandes der AWO an die Caritas in Danzig geflossen , " damit dort ein Altenheim gebaut wird " .
" Wenn da was gebucht worden ist , dann ist das nicht in Ordnung " - höchstens einen Buchungsfehler kann sie sich vorstellen . 
Die ehrenamtliche Landesvorsitzende Wedemeier weiß von diesem Vorgang nichts , " ich kontrolliere solche Sachen doch nicht , das machen die hauptamtlichen Geschäftsführer . " 
Kontrolliert werden die Geschäftsführer von den gewählten Revisoren des AWO-Landesverbandes , das sind Detlev Griesche und Karin Freudenthal . 
Da der Landesverband als Dachverband ohne hauptamtliches Personal nur einen " ganz kleinen Haushalt " hat ( Wedemeier ) , hätten Summen von Hapaq Lloyd oder 165.000 Mark schon auffallen müssen . 
      

Text segment:

Veruntreute die AWO Spendengeld ? 
Staatsanwaltschaft muß AWO-Konten prüfen
Flossen 165.000 Mark Sammelgelder für Flutopfer in ein Altenheim in Danzig ? 
Landesvorsitzende Ute Wedemeier : Ein Buchungsfehler 
Im Januar hat die Arbeiterwohlfahrt Bremen ihren langjährigen Geschäftsführer Hans Taake fristlos entlassen , nun wird auch der Vorstand der Wohlfahrtsorganisation in den Fall hineingzogen . 
In einer anonymen Anzeige werden der Bremer Staatsanwaltschaft Details über dubiose finanzielle Transaktionen mitgeteilt . 
Verantwortlich , so das Schreiben einer Mitarbeiterin der AWO , sei die Landesvorsitzende Uter Wedemeier , die sich jetzt als " Sauberfrau " gebe , " wo doch alle wissen , wie eng sie mit Taake zusammenhing " . 
Vorwurf Nummer 1 : 165.000 Mark aus der bundesweiten Geldsammlung für die Flutopfer in Südpolen seien über das Konto des Bremer Landesverbandes der AWO an die Caritas in Danzig geflossen , " damit dort ein Altenheim gebaut wird " . 
Das Altenheim sei " ein Prestigeobjekt von ihr und anderen " . 
In der Tat sitzt Ute Wedemeier im Kuratorium für das Altenheim , eine derartige Umleitung von Geldern habe es aber nicht gegeben , sagt sie . 
" Wenn da was gebucht worden ist , dann ist das nicht in Ordnung " - höchstens einen Buchungsfehler kann sie sich vorstellen . 
Volker Tegeler , stellvertretender Geschäftsführer des Landesverbandes , sagt : 
" Es gibt so eine Buchung . " 
In einer internen Kontrolle nach der Kündigung von Taake sei dies aufgefallen , zur Aufklärung solle ein externer Wirtschaftsprüfer beauftragt werden . 
Verantwortlich für die Finanzen des Landesverbandes sei aber " durchgehend Herr Taake " gewesen , sagt Tegeler . 
Aufgefallen bei der internen Prüfung ist auch Vorwurf Nummer 2 : Die AWO hat sich für Seniorenreisen nach Mallorca von Hapaq Lloyd Provisionen zahlen lassen . 
Die seien auf ein Konto des Landesverbandes der AWO geflossen , weil sie dort vor einer Finanzamtsprüfung sicherer gewesen seien . 
Tegeler bestätigt den Vorgang der Provisionszahlungen , meint allerdings , es müsse ein " Buchungsfehler " gewesen sein . 
Die ehrenamtliche Landesvorsitzende Wedemeier weiß von diesem Vorgang nichts , " ich kontrolliere solche Sachen doch nicht , das machen die hauptamtlichen Geschäftsführer . " 
Kontrolliert werden die Geschäftsführer von den gewählten Revisoren des AWO-Landesverbandes , das sind Detlev Griesche und Karin Freudenthal . 
Freuden-thal wollte gestern nichts dazu sagen , ob bei ihren Prüfungen ihr etwas aufgefallen sei . 
Da der Landesverband als Dachverband ohne hauptamtliches Personal nur einen " ganz kleinen Haushalt " hat ( Wedemeier ) , hätten Summen von Hapaq Lloyd oder 165.000 Mark schon auffallen müssen . 
Vorwurf Nummer 3 : Die Landesvorsitzende Ute Wedemeier hatte auf AWO-Kosten ein Handy . 
" Hier werden Beiträge von kleinen Leuten veraast , die von ehrenamtlichen Kassierern fünf Mark weise gesammelt werden " , schreibt die anonyme AWO-Mitarbeiterin an die Staatsanwaltschaft . 
Obwohl Frau Wedemeier " vor allem Privatgespräche über das Handy " führe , würde alles von der AWO bezahlt . 
Ute Wedemeier hält es für " selbstverständlich " , daß sie als ehrenamtliche Vorsitzende ein dienstliches Handy hat . 
Insbesondere wegen ihrer Aktivitäten in Riga und Danzig müsse sie erreichbar sein und auch telefonieren können . 
Wieviel da monatlich fällig wird , weiß sie aber nicht - " die Rechnung geht direkt an die AWO " . 
Hintergrund der gegenseitigen Vorwürfe in der Arbeiterwohlfahrt sind offenbar scharfe Konkurrenzen zwischen Bremern und Bremerhavenern . 
Als es in dieser Woche um die Neubesetzung des ehrenamtlichen Geschäftsführer-Postens im Landesverbandes ging , da sind diese Differenzen wieder aufgebrochen . 
Lothar Koring , Bremerhavener AWO-Vorsitzender , wollte seinen Bremerhavener Geschäftsführer Volker Tegeler auch im Landesverband zum Geschäftsführer machen . 
Koring selbst hatte früher auch gegen Ute Wedemeier für den Landesvorsitz kandidiert . 
Gegen Tegeler sprach allerdings , daß noch ein staatsanwaltschaftliches Ermittlungsverfahren gegen ihn läuft . 
Und Koring war früher einmal in schiefes Licht geraten , weil er bei einer Prüfgesellschaft im Vorstand war , die die AWO , wo er Kreisvorsitzender ist , prüfte . 
Seine Position bei der Prüfgesellschaft mußte er damals niederlegen , den AWO-Posten nicht . 
K.W.

### Parlament debattes

Texts of C. Blocher, only words with initial caps are used (mostly nouns)
* what are the topics
* what are the most interesting statements of these topics

* read the text 
* for each topic
    * output the 3 most topic related sentences
   

In [ ]:
import codecs,re

origLine={}
blocherDoc=[]
docId=0
with codecs.open('blocher.tsv',"r") as f:
     for line in f:
        party,speaker,text=line.strip().split('\t')  
        newtext= re.findall('[A-Z][a-zA-Züäö]+', text) # keep nouns as a list
        if len(newtext) <3 :
            continue
        newline = ' '.join(newtext)  # produce a string
        origLine[text]=newline       # index it
        blocherDoc.append(newline)

#blocherDoc #=blocherDoc[:20]

In [ ]:
no_features=5000
no_topics = 20

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=3, max_features=no_features)

tfidf = tfidf_vectorizer.fit_transform(blocherDoc)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

vis_nmf = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
vis_nmf

In [ ]:
#W=nmf.fit_transform(tfidf)
#H=nmf.components_
display_topics(nmf, tfidf_feature_names, no_top_words)

In [ ]:
def bestSentencesOfTopic2(doc,H,index,lines):
    """ lines=number of sentences 
        index=topic id
    """
    simIndexed={}
    for i in range(0,lines):
        if norm(X)==0:
            simIndexed[i]=0
            continue
        sim=np.dot(X,H[index])/(norm(X)*norm(H[index]))
        simIndexed[i]=np.array(sim)[0].tolist()[0]  
    return simIndexed


def getTopic2(X,H,topicLen):
    """ which topic is most similar to X (a tfidf representation of a document)"""
    bestsim=0
    index=0
    for i in range(0,topicLen-1):
        sim=np.dot(X,H[i])/(norm(X)*norm(H[i]))
        if sim>bestsim:
            bestsim=sim
            index=i
    return index

In [ ]:
W=nmf.fit_transform(tfidf)
H=nmf.components_

docId=0
sentOfTopic={}
    
for X in tfidf:
    X=X[0].toarray()    
    index=getTopic(X,H,no_topics)  
    sim=np.dot(X,H[index])/(norm(X)*norm(H[index]))
    if index in sentOfTopic:
        sentOfTopic[index].append(docId)
    else:
        sentOfTopic[index]=[docId]
    docId+=1
    
for t in sentOfTopic:
    simIndexed={}
    bestsim=0
    if t==1:
        print(sentOfTopic[t])
        break
    for d in sentOfTopic[t]:
        if d==1185:
            print(t)
            
        X=tfidf[d]
        X=X[0].todense()
        sim=np.dot(X,H[t])/(norm(X)*norm(H[t]))
        sim=np.array(sim)[0].tolist()[0]
        if sim > bestsim:
            bestsim=sim
            doc=d
    print(t,doc,bestsim)
    print(blocherDoc[doc])


In [ ]:
# svp
origLine={}
svpDoc=[]
docId=0
with codecs.open('svp.tsv',"r") as f:
     for line in f:
        party,speaker,text=line.strip().split('\t')  
        newtext= re.findall('[A-Z][a-zA-Züäö]+', text) # keep nouns as a list
        newline = ' '.join(newtext)  # produce a string
        if len(newtext) <3 :
            continue
        origLine[text]=newline      # index it
        svpDoc.append(newline)

svpDoc=removeDocs(blocherDoc)

In [ ]:
no_features=5000
no_topics = 20

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=3, max_features=no_features)

tfidf = tfidf_vectorizer.fit_transform(svpDoc)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

W=nmf.fit_transform(tfidf)
H=nmf.components_

vis_nmf = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
vis_nmf


In [ ]:
# sp
origLine={}
spDoc=[]
docId=0
with codecs.open('svp.tsv',"r") as f:
     for line in f:
        party,speaker,text=line.strip().split('\t')  
        newtext= re.findall('[A-Z][a-zA-Züäö]+', text) # keep nouns as a list
        newline = ' '.join(newtext)  # produce a string
        if len(newtext) <3 :
            continue
        origLine[text]=newline      # index it
        spDoc.append(newline)

spDoc=removeDocs(blocherDoc)

In [ ]:
no_features=5000
no_topics = 20

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=3, max_features=no_features)

tfidf = tfidf_vectorizer.fit_transform(blocherDoc)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

W=nmf.fit_transform(tfidf)
H=nmf.components_

vis_nmf = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
vis_nmf

### English texts


In [ ]:
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 20

# Run NMF
#nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf)


In [ ]:
### from sklearn.decomposition import NMF
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()

#pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

vis_nmf = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)
vis_nmf